To measure model performance we will use the test portion of the data and determine stats including ROC curve

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# installations
!pip uninstall -y tensorflow keras

Uninstalling Keras-2.2.4:
  Successfully uninstalled Keras-2.2.4


In [ ]:
!nvcc --version
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:03_CDT_2017
Cuda compilation tools, release 9.0, V9.0.176
--2021-05-02 11:36:09--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.16.29
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.16.29|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb [following]
--2021-05-02 11:36:10--  https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Pro

In [ ]:
# installations
!pip install -q tensorflow-gpu==1.13.1 keras==2.3.0 scikit-learn==0.20.0 pandas==0.25.0 numpy==1.16.0 keras-vis
!pip install opencv-python


     |████████████████████████████████| 378kB 14.0MB/s 
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.


In [ ]:
!pip install pillow
!pip install scipy==1.1.0

In [ ]:
# imports
from __future__ import print_function
import numpy as np
import pandas as pd
import time
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras import losses
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import model_from_json
from keras import backend as K
from keras.layers import Input, Flatten, Dense, Dropout, AveragePooling2D # Dropout, Convolution2D, Flatten, MaxPool2D
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from collections import Counter
from keras import regularizers

Using TensorFlow backend.


In [ ]:
def ShowImage(im, title='', ax=None):
  if ax is None:
    plt.figure()
  plt.axis('off')
  im = ((im + 1) * 127.5).astype(np.uint8)
  plt.imshow(im)
  plt.title(title)

def ShowHeatMap(im, title='', ax=None):
  if ax is None:
    plt.figure()
  plt.axis('off')
  plt.imshow(im, cmap='inferno')
  plt.title(title)

In [ ]:
# inference



modelFile = '/content/drive/My Drive/worm_models/movement/models/3d/200130/Feb-03-2020_16-22_worm_MobileNet_fullres.json'
weightsFile = '/content/drive/My Drive/worm_models/movement/models/3d/200130/weights.125-0.30.hdf5'

img_width, img_height = 900, 900
batchSize = 10
testSamplesNumber = 20

with open(modelFile, 'r', encoding='utf-8') as f:
  model = model_from_json(f.read())
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])
model.load_weights(weightsFile, by_name=False)
model.save('/content/drive/My Drive/worm_models/movement/models/3d/200130/model.h5')


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 150, 150, 96)      11712     
_________________________________________________________________
spatial_dropout2d_7 (Spatial (None, 150, 150, 96)      0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 150, 150, 96)      384       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 75, 75, 96)        0         
_________________________________________________________________
spatial_dropout2d_8 (Spatial (None, 75, 75, 96)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 71, 71, 256)       614656    
_________________________________________________________________
spatial_dropout2d_9 (Spatial (None, 71, 71, 256)      

In [ ]:
import cv2
from PIL import Image
from vis.utils  import utils
from vis.visualization import visualize_cam #, visualize_saliency, get_num_filters
import re

def plotNSave(im,file,grads):
    f, ax = plt.subplots(1, 3)
    ax[0].imshow(im, alpha=1)
    ax[1].imshow(grads, cmap='jet')
    ax[2].imshow(grads, cmap='jet')
    ax[2].imshow(im, alpha=0.5)
    f.savefig(file, bbox_inches='tight')

def listImages(folder, endPattern):
    data = list()
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(endPattern):
                 data.append(file)
    return data

def checkDir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        
testFolder = '/content/drive/My Drive/worm_models/movement/test_images/3d/low'
images = listImages(testFolder,".png")
print(images)
#imageFolder = os.path.join(testFolder,'CAM')
#checkDir(imageFolder)          
#layer_idx = utils.find_layer_idx(model,'avg_pool')
plt.rcParams['figure.figsize']=(18,6)

for image in images:
    imgPath = os.path.join(testFolder,image)
    im = cv2.resize(cv2.imread(imgPath, cv2.IMREAD_GRAYSCALE), (img_width, img_height))
    im = np.round(im.astype(float)*2.6).astype(int).reshape(img_width, img_height, 1)
    #plt.imshow(im)
    plt.figure()
    modifier = 'guided'
    #modifier = 'relu'
    #grads = visualize_saliency(model, layer_idx, filter_indices=0, seed_input=im, backprop_modifier=modifier)
    grads = visualize_cam(model, layer_idx=17, filter_indices=None, seed_input=im, backprop_modifier=modifier)
    file_name = re.sub(r"png", "pdf",imgPath)
    plotNSave(np.squeeze(im),file_name,grads)
    cv2.imwrite(file_name.replace('.pdf','_cam.tif'),grads)

In [ ]:
model.layers[17]

In [ ]:
im.reshape(900, 900, 1).shape

In [ ]:
testFolder = '/content/drive/My Drive/worm_models/movement/test_images/3d/high'
images = listImages(testFolder,".png")
print(images)
#imageFolder = os.path.join(testFolder,'CAM')
#checkDir(imageFolder)          
#layer_idx = utils.find_layer_idx(model,'avg_pool')
plt.rcParams['figure.figsize']=(18,6)

for image in images:
    imgPath = os.path.join(testFolder,image)
    im = cv2.resize(cv2.imread(imgPath, cv2.IMREAD_GRAYSCALE), (img_width, img_height))
    im = np.round(im.astype(float)*2.6).astype(int).reshape(img_width, img_height, 1)
    #plt.imshow(im)
    plt.figure()
    modifier = 'guided'
    #modifier = 'relu'
    #grads = visualize_saliency(model, layer_idx, filter_indices=0, seed_input=im, backprop_modifier=modifier)
    grads = visualize_cam(model, layer_idx=17, filter_indices=None, seed_input=im, backprop_modifier=modifier)
    file_name = re.sub(r"png", "pdf",imgPath)
    plotNSave(np.squeeze(im),file_name,grads)
    cv2.imwrite(file_name.replace('.pdf','_cam.tif'),grads)